<a href="https://colab.research.google.com/github/ApoorvaNagarajan/Image-Filter-Kernels/blob/master/assignment16/assignment16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing all python packages

In [1]:
!pip install -q keras
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import SGD
from google.colab import drive
import tensorflow as tf
from keras import backend as k
import numpy as np
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


**clone the utility repo created**

In [2]:
!git clone https://github.com/ApoorvaNagarajan/DNN_utils.git

from DNN_utils import tfrecord_utils

Cloning into 'DNN_utils'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 31 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


**Mount google drive to fetch the CIFAR dataset stored in tfRecord format**

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


**Configuration**

In [0]:
# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters

BATCH_SIZE = 128 #@param {type:"number"}
EPOCHS = 50 #@param {type:"number"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.1 #@param {type:"number"}
WEIGHT_DECAY = 1e-4 #@param {type:"number"}
l = 12 #@param {type:"number"}
num_filter = 12 #@param {type:"number"}
compression = 0.5 #@param {type:"number"}
dropout_rate = 0.2 #@param {type:"number"}



In [0]:
#Configuration

NUM_CLASSES = 10 
IMG_SHAPE=[-1, 32, 32, 3]
NUM_TRAIN_IMG = 50000
NUM_TEST_IMG = 10000
BATCHES_PER_EPOCH = NUM_TRAIN_IMG//BATCH_SIZE + 1

#Load tfRecord format CIFAR10 dataset

In [0]:
train_x, train_y = tfrecord_utils.parse_tfRecord('/content/drive/My Drive/datasets/cifarTfTrain.tfrecords', NUM_TRAIN_IMG, BATCH_SIZE, IMG_SHAPE, NUM_CLASSES)
test_x, test_y = tfrecord_utils.parse_tfRecord('/content/drive/My Drive/datasets/cifarTfTest.tfrecords', NUM_TEST_IMG, BATCH_SIZE, IMG_SHAPE, NUM_CLASSES)

#Define model

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(NUM_CLASSES, activation='softmax')(flat)
    
    return output

#Create network and run

In [0]:
input = Input(shape=(32, 32, 3), tensor=train_x)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [17]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 12)   324         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 12)   48          conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 12)   0           batch_normalization_53[0][0]     
____________________________________________________________________________________________

In [0]:
def scheduler(epoch, lr):
  return np.interp(epoch, [0, (EPOCHS+1)//2, ((EPOCHS+1)*3)//4, EPOCHS], [LEARNING_RATE, LEARNING_RATE/10,LEARNING_RATE/100, LEARNING_RATE/100])

#checkpoint = ModelCheckpoint("assign16_model",verbose=1,save_best_only=True, monitor='val_acc', mode='max')

In [19]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=LEARNING_RATE,momentum=MOMENTUM,nesterov=True),
              metrics=['accuracy'],
             target_tensors=[train_y])

W0902 18:06:28.558279 140528108038016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [20]:
model.fit(epochs=EPOCHS,
          steps_per_epoch=BATCHES_PER_EPOCH,
          verbose=1,
          validation_data=(test_x, test_y),
          validation_steps=NUM_TEST_IMG//BATCH_SIZE,
          callbacks=[LearningRateScheduler(scheduler, verbose=0)])

W0902 18:06:33.099559 140528108038016 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
391/391 [==============================] - 187s 478ms/step - loss: 1.7010 - acc: 0.3727 - val_loss: 2.0529 - val_acc: 0.3452
Epoch 2/50
391/391 [==============================] - 165s 423ms/step - loss: 1.2772 - acc: 0.5349 - val_loss: 2.0587 - val_acc: 0.4555
Epoch 3/50
391/391 [==============================] - 165s 423ms/step - loss: 1.1034 - acc: 0.6024 - val_loss: 1.1655 - val_acc: 0.6083
Epoch 4/50
391/391 [==============================] - 165s 423ms/step - loss: 1.0075 - acc: 0.6386 - val_loss: 1.3474 - val_acc: 0.5808
Epoch 5/50
391/391 [==============================] - 165s 423ms/step - loss: 0.9424 - acc: 0.6643 - val_loss: 1.1269 - val_acc: 0.6368
Epoch 6/50
391/391 [==============================] - 166s 423ms/step - loss: 0.8754 - acc: 0.6878 - val_loss: 1.6894 - val_acc: 0.5536
Epoch 7/50
391/391 [==============================] - 166s 423ms/step - loss: 0.8243 - acc: 0.7065 - val_loss: 0.9039 - val_acc: 0.6994
Epoch 8/50
391/391 [============================